# ASSIGNATIONS TAXONOMIQUES

## CHARGEMENT DES PACKAGES

In [3]:
library(Rcpp)
library(dada2)
library(Biostrings)
library(tidyverse) 

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    I, expand.grid, unname


Loading required package: IRanges

Loading required package: XVector

Loading required package: GenomeInfoDb


Attaching pack

## OUVERTURE DES SEQUENCES

In [4]:
# ouverture du fichier .fasta 
seq_rdp <- readDNAStringSet("data/sequences.fasta", format="fasta",
               nrec=-1L, skip=0L, seek.first.rec=FALSE,
               use.names=TRUE, with.qualities=FALSE)
seq_rdp


DNAStringSet object of length 8130:
       width seq                                            names               
   [1]   253 TACGGAGGGTGCGAGCGTTAAT...AAAGCGTGGGGAGCAAACAGG f41eb548cba466f8a...
   [2]   253 TACGGAGGGTCCAAGCGTTATC...AAAGCGTGGGGAGCGAACGGG dda1c8a4aafcc6c3d...
   [3]   253 TACGGAGGGTGCGAGCGTTGTC...AAAGCATGGGGAGCGAACAGG 394a72d1e51d2532c...
   [4]   253 TACGAAGGGGGCTAGCGTTGTT...AAAGTATGGGGAGCAAACAGG 80b063178d4c8e004...
   [5]   253 TACGGAGGGTGCAAGCGTTAAT...AAAGCGTGGGGAGCAAACAGG d46e2205f0c6ecf67...
   ...   ... ...
[8126]   253 TACGGAGGGTGCGAGCGTTAAT...AAAGCGTGGGGAGCAAACAGG 1807bf5620ecb781f...
[8127]   253 TACGGAGGATCCAAGCGTTATC...AAAGCGTGGGGAGCAAACCGG 96d060050dcb63953...
[8128]   253 TACGGAGGATCCAAGCGTTATC...AAAGCGTGGGTAGCGAACAGG 3ce45e39df4224347...
[8129]   253 TACGGAGGGTGCGAGCGTTAAT...AAAGCGTGGGGAGCAAACAGG 3cf6e306e475ab9f4...
[8130]   253 TACAGAGGGGGCAAGCGTTGTT...AAAGCCAGGGGAGCAAACGGG b35e30cf9079e8eae...

In [5]:
# transformation en vecteurs de charactères, necessaire pour la fonction assignTaxonomy
seq_rdp_char <- as.character(seq_rdp)
head(seq_rdp_char)

f41eb548cba466f8a672f4651a674958 
"TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGCATGCAGGTGGTTTGTTAAGTCAGATGTGAAAGCCCGGGGCTCAACCTCGGAAGGTCATTTGAAACTGGCAAACTAGAGTACTGTAGAGGGGGGTAGAATTTCAGGTGTAGCGGTGAAATGCGTAGAGATCTGAAGGAATACCAGTGGCGAAGGCGGCCCCCTGGACAGATACTGACACTCAGATGCGAAAGCGTGGGGAGCAAACAGG" 
                                                                                                                                                                                                                               dda1c8a4aafcc6c3d1a4ae4b7bd73102 
"TACGGAGGGTCCAAGCGTTATCCGGAATCATTGGGTTTAAAGGGTTCGTAGGCGGGCTAATTAGTCAGAGGTGAAATCCAGCAGCTTAACTGTTGAACTGCCTTTGATACTGTTAGTCTTGAGTTATATGGAAGTAGATAGAATATGTAGTGTAGCGGTGAAATGCATAGATATTACATAGAATACCGATTGCGAAGGCAGTCTACTACGTATATACTGACGCTGATGAACGAAAGCGTGGGGAGCGAACGGG" 
                                                                                                                                                                                                                               394a72d1e51d2532c78dc8fdafbd0408 
"TACGGAGGGTGCGAGCGTTGTCCGGATTTATTGGGTTTAAAGGGTGCGTAGGTGGCTTGATAAGTCAGTGGTGAAATACATCCGCTTAACGGATGAGGTGCCATTGATACTGTTAAGCTTGAAAGAATTGGAGGCTGCCGGAATGGATGGTGTAGCGGTGAAATGCATAGATATCATCCAGAACACCGATTGCGAAGGCAGGTGGCTACGATTTATTTGACACTGAGGCACGAAAGCATGGGGAGCGAACAGG" 
                                                                                                                                                                                                                               80b063178d4c8e0047bb2d0653d300fd 
"TACGAAGGGGGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGGGTGCGTAGGCTGTTTTGTAAGTTAGATGTGAAAGCCTCGGTTTCAAGCTGAGAATAGCATTTAATACTGCAAGACTAGAGGATAAGAGAGGACAGGGGAATTCTGAGTGTAGAGGTAAAATTCGTAGATATTCAGAGGAACACCGGAGGCGAAAGCGCCTGTCTGGATTAATCCTGACGCTGAGGCACGAAAGTATGGGGAGCAAACAGG" 
                                                                                                                                                                                                                               d46e2205f0c6ecf67b51f83d111c509c 
"TACGGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCACGCAGGCGGTTTGTTAAGTCAGATGTGAAATCCCCGGGCTCAACCTGGGAACTGCATCTGATACTGGCAAGCTTGAGTCTCGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGAATACCGGTGGCGAAGGCGGCCCCCTGGACGAAGACTGACGCTCAGGTGCGAAAGCGTGGGGAGCAAACAGG" 
                                                                                                                                                                                                                               d7c175c5b206160e3c14ffd3ae2019d7 
"TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTACGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTGTGATAGAGGGTGGTAGAATTTCAGGTGTAGCGGTGAAATGCGTAGAGATCTGAAGGAATACCGATGGCGAAGGCAGCCACCTGGGTCAACACTGACGCTCATGTACGAAAGCGTGGGGAGCAAACGGG"

## ASSIGNATION TAXONOMIQUE SELON LES DATABASES

## 1) RDP

In [6]:
# Assignation taxonomique en utilisant la base de référence RDP
taxa <- assignTaxonomy(seq_rdp_char,"data/TaxoRef_Training/rdp_19_toSpecies_trainset.fa.gz", multithread=TRUE)
head(taxa)

# Convertir la matrice taxa en data frame
taxa_RDP <- as.data.frame(taxa)
taxa_RDP <- rownames_to_column(taxa_RDP, var="sequence")
                                 
# Ajouter une colonne avec les noms de seq_rdp_char
taxa_RDP$ASV <- names(seq_rdp_char)

# Ajouter une colonne "Data base" avec la valeur "RDP" pour toutes les lignes
taxa_RDP$Data_base <- "RDP"

# supprimer la colonne séquence

taxa_RDP$sequence <- NULL
head(taxa_RDP)

,Kingdom,Phylum,Class,Order,Family,Genus,Species
TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGCATGCAGGTGGTTTGTTAAGTCAGATGTGAAAGCCCGGGGCTCAACCTCGGAAGGTCATTTGAAACTGGCAAACTAGAGTACTGTAGAGGGGGGTAGAATTTCAGGTGTAGCGGTGAAATGCGTAGAGATCTGAAGGAATACCAGTGGCGAAGGCGGCCCCCTGGACAGATACTGACACTCAGATGCGAAAGCGTGGGGAGCAAACAGG,Bacteria,Pseudomonadota,Gammaproteobacteria,Vibrionales,Vibrionaceae,Vibrio,NA
TACGGAGGGTCCAAGCGTTATCCGGAATCATTGGGTTTAAAGGGTTCGTAGGCGGGCTAATTAGTCAGAGGTGAAATCCAGCAGCTTAACTGTTGAACTGCCTTTGATACTGTTAGTCTTGAGTTATATGGAAGTAGATAGAATATGTAGTGTAGCGGTGAAATGCATAGATATTACATAGAATACCGATTGCGAAGGCAGTCTACTACGTATATACTGACGCTGATGAACGAAAGCGTGGGGAGCGAACGGG,Bacteria,Bacteroidota,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Wenyingzhuangia,gracilariae
TACGGAGGGTGCGAGCGTTGTCCGGATTTATTGGGTTTAAAGGGTGCGTAGGTGGCTTGATAAGTCAGTGGTGAAATACATCCGCTTAACGGATGAGGTGCCATTGATACTGTTAAGCTTGAAAGAATTGGAGGCTGCCGGAATGGATGGTGTAGCGGTGAAATGCATAGATATCATCCAGAACACCGATTGCGAAGGCAGGTGGCTACGATTTATTTGACACTGAGGCACGAAAGCATGGGGAGCGAACAGG,Bacteria,Bacteroidota,Cytophagia,Cytophagales,Spirosomaceae,Dyadobacter,jejuensis
TACGAAGGGGGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGGGTGCGTAGGCTGTTTTGTAAGTTAGATGTGAAAGCCTCGGTTTCAAGCTGAGAATAGCATTTAATACTGCAAGACTAGAGGATAAGAGAGGACAGGGGAATTCTGAGTGTAGAGGTAAAATTCGTAGATATTCAGAGGAACACCGGAGGCGAAAGCGCCTGTCTGGATTAATCCTGACGCTGAGGCACGAAAGTATGGGGAGCAAACAGG,Bacteria,Pseudomonadota,Alphaproteobacteria,NA,NA,NA,NA
TACGGAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGCACGCAGGCGGTTTGTTAAGTCAGATGTGAAATCCCCGGGCTCAACCTGGGAACTGCATCTGATACTGGCAAGCTTGAGTCTCGTAGAGGGGGGTAGAATTCCAGGTGTAGCGGTGAAATGCGTAGAGATCTGGAGGAATACCGGTGGCGAAGGCGGCCCCCTGGACGAAGACTGACGCTCAGGTGCGAAAGCGTGGGGAGCAAACAGG,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia/Shigella,coli
TACGGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTACGCAGGCGGTTTGTTAAGCGAGATGTGAAAGCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTGTGATAGAGGGTGGTAGAATTTCAGGTGTAGCGGTGAAATGCGTAGAGATCTGAAGGAATACCGATGGCGAAGGCAGCCACCTGGGTCAACACTGACGCTCATGTACGAAAGCGTGGGGAGCAAACGGG,Bacteria,Pseudomonadota,Gammaproteobacteria,Alteromonadales,Pseudoalteromonadaceae,Pseudoalteromonas,NA


,Kingdom,Phylum,Class,Order,Family,Genus,Species,ASV,Data_base
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Bacteria,Pseudomonadota,Gammaproteobacteria,Vibrionales,Vibrionaceae,Vibrio,NA,f41eb548cba466f8a672f4651a674958,RDP
2,Bacteria,Bacteroidota,Flavobacteriia,Flavobacteriales,Flavobacteriaceae,Wenyingzhuangia,gracilariae,dda1c8a4aafcc6c3d1a4ae4b7bd73102,RDP
3,Bacteria,Bacteroidota,Cytophagia,Cytophagales,Spirosomaceae,Dyadobacter,jejuensis,394a72d1e51d2532c78dc8fdafbd0408,RDP
4,Bacteria,Pseudomonadota,Alphaproteobacteria,NA,NA,NA,NA,80b063178d4c8e0047bb2d0653d300fd,RDP
5,Bacteria,Pseudomonadota,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia/Shigella,coli,d46e2205f0c6ecf67b51f83d111c509c,RDP
6,Bacteria,Pseudomonadota,Gammaproteobacteria,Alteromonadales,Pseudoalteromonadaceae,Pseudoalteromonas,NA,d7c175c5b206160e3c14ffd3ae2019d7,RDP


EXPORTATION TABLEAU

In [24]:
# write.table(taxa_table, file= "data/output/assign_RDP.csv", sep= ";" , row.names= FALSE, col.names= TRUE)

## 2) SILVA

In [1]:
# Assignation taxonomique en utilisant la base de référence Silva
taxa <- assignTaxonomy(seq_rdp_char, "data/TaxoRef_Training/silva_nr99_v138.2_toSpecies_trainset.fa.gz", multithread=TRUE)
head(taxa)

# Convertir la matrice taxa en data frame
taxa_SILVA <- as.data.frame(taxa)
taxa_SILVA <- rownames_to_column(taxa_SILVA, var="sequence")
                                 
# Ajouter une colonne avec les noms de seq_rdp_char
taxa_SILVA$ASV <- names(seq_rdp_char)

# Ajouter une colonne "Data base" avec la valeur "RDP" pour toutes les lignes
taxa_SILVA$Data_base <- "RDP"

# supprimer la colonnes séquence

taxa_SILVA$sequence <- NULL
head(taxa_SILVA)

ERROR: Error in assignTaxonomy(seq_rdp_char, "data/TaxoRef_Training/silva_nr99_v138.2_toSpecies_trainset.fa.gz", : could not find function "assignTaxonomy"


EXPORTATION TABLEAU

In [ ]:
# write.table(taxa_SILVA, file= "data/output/assign_SILVA.csv", sep= ";" , row.names= FALSE, col.names= TRUE)

## 3) GG2

In [ ]:
# Assignation taxonomique en utilisant la base de référence Gg2
taxa_gg2 <- assignTaxonomy(seq_rdp_char, "data/TaxoRef_Training/gg2_2024_09_toSpecies_trainset.fa.gz", multithread=TRUE)
head(taxa_gg2)

# Convertir la matrice taxa en data frame
taxa_gg2 <- as.data.frame(taxa)
taxa_gg2 <- rownames_to_column(taxa_gg2, var="sequence")
                                 
# Ajouter une colonne avec les noms de seq_rdp_char
taxa_gg2$ASV <- names(seq_rdp_char)

# Ajouter une colonne "Data base" avec la valeur "RDP" pour toutes les lignes
taxa_gg2$Data_base <- "RDP"

# supprimer la colonnes séquence

taxa_gg2$sequence <- NULL
head(taxa_gg2)

EXPORTATION TABLEAU 

In [ ]:
# write.table(taxa_gg2, file= "data/output/assign_gg2.csv", sep= ";" , row.names= FALSE, col.names= TRUE)